* 将 11 月预测销量填入训练集
* 直接用来预测 12 月销量走一波
* 测试结果是 LGBoost/XGBoost/GBDT, 其中尤以 LGB 最为关键

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [36]:
train = pd.read_csv('../../raw/LiChuan/trainSaleDate.csv')
test_A = pd.read_csv('../../raw/LiChuan/test_feature.csv')
test_B = pd.read_csv('../../raw/CarsSaleForecast/yancheng_testB_20180224.csv')
trainFeature = pd.read_csv('../../raw/LiChuan/train_feature.csv')
pred_Nov = pd.read_csv('../../raw/Submissions/result_180224_baseline*0.42_lightGBM*0.58.csv')

# 先拿基础数据

In [37]:
# 得到 2017-11 之前训练集
train = pd.concat([train, test_A]).reset_index(drop=True)
train.fillna(0, inplace=True)

In [39]:
# 用 RMSE=116 的数值填充 11 月数据
pred_Nov = pred_Nov['predict_quantity'].values
train['sale_quantity'][-140:] = pred_Nov

In [41]:
test_B['year'] = test_B['predict_date'].apply(lambda x: int(str(x)[0:4]))
test_B['month'] = test_B['predict_date'].apply(lambda x: int(str(x)[4:6]))

In [44]:
sellmonth = train[['class_id', 'how_many_month_has_sell', 'sale_quantity']][(train['year']==2017)&(train['month']==11)]
sellmonth.dropna(inplace=True) 
sellmonth['how_many_month_has_sell'] = sellmonth['how_many_month_has_sell']+1
sale_quantity_total_last_month = sellmonth['sale_quantity'].sum()
sellmonth.rename(columns={'sale_quantity': 'sale_quantity_this_last_month'}, inplace=True)

sellYear = train[['class_id', 'sale_quantity']][((train['year']==2016)&(train['month']>=12))|((train['year']==2017)&(train['month']<12))]
sellYear.dropna(inplace=True)
sellYear = sellYear.groupby('class_id').agg('sum').reset_index()
sale_quantity_total_last_year = sellYear['sale_quantity'].sum()
sellYear.rename(columns={'sale_quantity': 'sale_quantity_this_last_year'}, inplace=True)

test_B = pd.merge(test_B, trainFeature, on='class_id', how='left')
test_B = pd.merge(test_B, sellmonth, on='class_id', how='left')
test_B = pd.merge(test_B, sellYear, on='class_id', how='left')

test_B.loc[:, 'sale_quantity_total_last_month'] = sale_quantity_total_last_month
test_B.loc[:, 'sale_quantity_total_last_year'] = sale_quantity_total_last_year
test_B.loc[:, 'this_month_sale_class_count'] = 140

test_B.sort_values(['class_id'])
test_B.fillna(0.0, inplace=True)

test_B = test_B.drop(['predict_quantity'], axis=1)

In [48]:
columns = train.drop(['sale_quantity'], axis=1).columns.tolist()
test_B['sale_date'] = test_B['predict_date']
test_B.drop(['predict_date'], axis=1)
test_B = test_B[columns]

In [51]:
# 得到整体训练集
train = pd.concat([train, test_B]).reset_index(drop=True)
train.fillna(0, inplace=True)

In [53]:
# 保存一个过程 csv
train.to_csv('../../raw/LiChuan/Dec_Basic_feature.csv', index=False)

In [59]:
# 检查一下看看, ok 没什么问题
train[(train['year'] == 2017)&(train['month']==11)]['how_many_month_has_sell'].sum()

5501.0

In [60]:
train[(train['year'] == 2017)&(train['month']==12)]['how_many_month_has_sell'].sum()

5641.0